# 哈希算法

## 定义及特性
**定义：**将任意长度的二进制明文串（输入）转化成固定长度的二进制（输出）。输出也常被称为哈希值、摘要或指纹。

### 特性1：输出确定

In [1]:
# 导入内置的哈希运算模块
import hashlib

In [2]:
# 创建一条字符串
string = "I love blockchain"

In [3]:
# 调用sha256对象计算哈希值
value = hashlib.sha256(string.encode())
print(value)
print(type(value))

<sha256 HASH object @ 0x00000000167D0468>
<class '_hashlib.HASH'>


In [4]:
# 返回字符串摘要
value.digest()
print("字符串摘要长度：%s" % len(value.digest()))

字符串摘要长度：32


In [5]:
# 哈希摘要字节长度
print("哈希摘要字节长度：%s" % value.digest_size)
# 返回十六进制字符串摘要
print("十六进制字符串摘要：%s" % value.hexdigest())
print("十六进制字符串摘要长度：%s" % len(value.hexdigest()))
# 内部块长度
print("内部块长度：%s" % value.block_size)

哈希摘要字节长度：32
十六进制字符串摘要：84228b032ad69ca54729156f864eda73421e493af5723073695a15fc6fa25293
十六进制字符串摘要长度：64
内部块长度：64


**输入确定：**只要输入和使用的哈希算法是固定的，无论计算多少次，得出的结果都是确定的

In [6]:
# 直接计算
hashlib.sha256(b'I love blockchain').hexdigest()

'84228b032ad69ca54729156f864eda73421e493af5723073695a15fc6fa25293'

In [7]:
# 生成哈希对象
h = hashlib.sha256()
# 调用update()方法
h.update(b'I love blockchain')
# 返回十六进制字符串摘要
h.hexdigest()

'84228b032ad69ca54729156f864eda73421e493af5723073695a15fc6fa25293'

### 特性2：正向快速
只要确定了输入明文和哈希算法，在有限的时间和有限的资源内就能计算得到哈希值。

### 特性3：逆向困难
如果已知输出哈希值，在有限的时间内得到输入明文是困难的，但并不表示不存在。对于一个256位的输出，理论上需要选择$2^{256}+1$个不同的输入明文，计算每个明文的哈希值，并检查它们的值是否相等，该计算是困难的。

### 特性4：雪崩效应
哈希函数对输入是相当敏感的，即使只对原始输入数据做一点改动，哈希函数的输出结果就会有很大不同。

In [8]:
s1 = "I love blockchain"
s2 = "i love blockchain"
v1 = hashlib.sha256(s1.encode())
v2 = hashlib.sha256(s2.encode())
print("v1的哈希值：%s" % v1.hexdigest())
print("v2的哈希值：%s" % v2.hexdigest())

v1的哈希值：84228b032ad69ca54729156f864eda73421e493af5723073695a15fc6fa25293
v2的哈希值：d2eb69267dcf9e10e489cf61ed6427944a99db170063f879ba150d63f79cc5df


### 特性5：避免碰撞
碰撞是指对于不同的输入，通过哈希算法计算得到相同的输出。目前SHA256算法发现碰撞的可能性小点。

#### 双SHA256双哈希算法

In [9]:
def sha256d(string):
    # 如果输入是字符串，则将其转化为字节串
    if not isinstance(string, bytes):
        string = string.encoding()
    
    # 首先计算输入的哈希摘要hashlib.sha256(stirng).digest()，为字节串类型
    # 然后计算哈希摘要的哈希值，输出为十六进制字符串
    return hashlib.sha256(hashlib.sha256(string).digest()).hexdigest()

## SHA256算法的一种实现
1. 填充明文：如果明文长度L小于$512n-64$，其中n为自然数，则L之后的第一位填充1，后面$512n-64-1-L$位填充0，最后64位无符号整数，用以表示明文填充前的长度。
2. 解析填充后的明文：将填充后的明文解析成N个512位分组。
3. 初始化缓存：先初始化8个长度为32位的字作为缓存abcdefgh，将最终的缓存转化为字节串或字符串（即信息摘要）。
4. 哈希过程：SHA256算法采用了6种逻辑函数，对明文的每个512位分组进行64步的迭代运算操作。每一步都以缓存abcdefg为输入，然后更新缓存，计算过程中的每一步都用到了一个常数$K_i$和一个32位的字$w_i$。  
6种逻辑函数如下：$$\begin{array}{ccc}
ch(x,y,z)=(x^{\wedge}y)\oplus(\bar{z}^{\wedge}x) \\
maj(x,y,z)=(x^{\wedge}y)\oplus(x^{\wedge}z)\oplus(y^{\wedge}z) \\
\Sigma_0(x)=ROTR^2(x)\oplus ROTR^{13}(x)\oplus ROTR^{22}(x) \\
\Sigma_1(x)=ROTR^6(x)\oplus ROTR^{11}(x)\oplus ROTR^{25}(x) \\
\sigma_0=ROTR^7(x)\oplus ROTR^{18}(x)\oplus SHR^{3}(x) \\
\sigma_1=ROTR^{17}(x)\oplus ROTR^{19}(x)\oplus SHR^{10}(x)
\end{array}$$
其中，$SHR^n(x)=x \gg n, ROTR^n(x)=(x \gg n) | (x \gg (32 - n))$  
(1)生成一个长度为64的空列表w，用于存储64个字。将512位明文转换为16个32位字，并赋予列表w的前16个元素，接下来按照下列公式填充列表w的后48个元素$$
\begin{array}{ccc}
\sigma_0=ROTR^7(w_{i-15})\oplus ROTR^{18}(w_{i-15})\oplus SHR^{3}(w_{i-15}) \\
\sigma_1=ROTR^{17}(w_{i-2})\oplus ROTR^{19}(w_{i-2})\oplus SHR^{10}(w_{i-2}) \\
w_i=\sigma_0+\sigma_1+w_{i-16}+w_{i-7}
\end{array}
$$其中，$15 \leq i \leq 63$。  
(2)更新缓存abcdefg：  
for i in 0 to 63 do$$\begin{array}{ccc}
\sigma_0=ROTR^7(a)\oplus ROTR^{18}(a)\oplus SHR^{3}(a) \\
maj(a,b,c)=(a^{\wedge}b)\oplus(a^{\wedge}c)\oplus(b^{\wedge}c) \\
T_2 = \sigma_0 + maj(a,b,c) \\
\sigma_1=ROTR^{17}(e)\oplus ROTR^{19}(e)\oplus SHR^{10}(e) \\
ch(e,f,g)=(e^{\wedge}f)\oplus(\bar{g}^{\wedge}e) \\
T_1 = h + \sigma_0 + ch(e,f,g) + K_i + w_i \\
h = g \\
g = f \\
f = e \\
e = d + T_1 \\
d = c \\
c = b \\
b = a \\
a = T_1 + T_2
\end{array}$$
5. 将最终得到的缓存abcdefg转换为字节串或字符串，得到信息摘要。

以下是Python的算法实现

In [10]:
# struct模块能实现Python值与C语言结构体的转换
import struct
# 实现二进制与ASCII码之间的转换
import binascii

In [11]:
# 64个常数K
_K = (0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5,
      0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
      0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3,
      0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
      0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc,
      0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
      0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7,
      0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
      0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13,
      0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
      0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3,
      0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
      0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5,
      0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
      0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208,
      0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2)

# 初始化缓存
_H = (0x6a09e667, 0xbb67ae85, 0x3c6ef372, 0xa54ff53a,
      0x510e527f, 0x9b05688c, 0x1f83d9ab, 0x5be0cd19)

In [12]:
# 定义sha_256类
class sha_256:
    # 输入参数为明文
    def __init__(self,m = None):
        # 初始化明文
        self.buffer = b''
        
        # 输入明文长度
        self.counter = 0
        self.H = _H
        self.K = _K

        if m:
            self.update(m)

    # 定义循环右移的方法
    def rotr(self, x, y):
        return ((x >> y) | (x << (32-y))) & 0xFFFFFFFF
    
    # 定义对单个分组进行操作的方法
    def operate(self,c):
        # 定义长度为64的空列表w
        w = [0]*64
        
        # 将单个分组转换为16个32位的字，并填充w列表的前16位 
        w[0:16] = struct.unpack('!16L', c)
        
        # 填充w列表的后48位
        for i in range(16, 64):
            s0 = self.rotr(w[i-15], 7) ^ self.rotr(w[i-15], 18) ^ (w[i-15] >> 3)
            s1 = self.rotr(w[i-2], 17) ^ self.rotr(w[i-2], 19) ^ (w[i-2] >> 10)
            w[i] = (w[i-16] + s0 + w[i-7] + s1) & 0xFFFFFFFF
        
        a,b,c,d,e,f,g,h = self.H
        
        # 执行64步迭代操作
        for i in range(64):
            s0 = self.rotr(a, 2) ^ self.rotr(a, 13) ^ self.rotr(a, 22)
            maj = (a & b) ^ (a & c) ^ (b & c)
            t2 = s0 + maj
            s1 = self.rotr(e, 6) ^ self.rotr(e, 11) ^ self.rotr(e, 25)
            ch = (e & f) ^ ((~e) & g)
            t1 = h + s1 + ch + self.K[i] + w[i]
            
            h = g
            g = f
            f = e
            e = (d + t1) & 0xFFFFFFFF
            d = c
            c = b
            b = a
            a = (t1 + t2) & 0xFFFFFFFF
        
        # 更新缓存
        self.H = [(x+y) & 0xFFFFFFFF for x,y in zip(self.H, [a,b,c,d,e,f,g,h])]

    # 定义更新N个分组缓存的方法    
    def update(self,m):
        if not m:
            return 
        
        # 获取明文
        self.buffer = m
        
        # 获取明文长度
        self.counter = len(m)
        
        # 计算明文长度表示的后64位
        length = struct.pack('!Q', int(self.counter*8))
        
        # 对前N-1个分组进行哈希过程
        while len(self.buffer) >= 64:
            self._operate(self.buffer[:64])
            self.buffer = self.buffer[64:]

        # 填充未处理的第N个分组至512位或1024位，并进行哈希过程    
        mdi = self.counter % 64
        
        # 如果第N个分组长度小于56，则填充至512位
        if mdi < 56:
            padlen = 55-mdi
            self.buffer += (b'\x80'+(b'\x00'*padlen) + length)
            self.operate(self.buffer)
        # 否则填充至1024位
        else:
            padlen = 119-mdi
            self.buffer += (b'\x80'+(b'\x00'*padlen) + length)
            for i in range(2):
                self.operate(self.buffer[i*64:(i+1)*64])
    
    # 输出明文摘要，字节串类型
    def digest(self):
        return struct.pack('!8L',*self.H)
    
    # 输出明文摘要，十六进制字符串类型
    def hexdigest(self):
        return binascii.hexlify(self.digest()).decode()

### 调用内置hashlib模块中的sha256算法进行对比

In [13]:
sha_256(b'111').digest()

b'\xf6\xe0\xa1\xe2\xacA\x94Z\x9a\xa7\xff\x8a\x8a\xaa\x0c\xeb\xc1*;\xcc\x98\x1a\x92\x9a\xd5\xcf\x81\n\t\x0e\x11\xae'

In [14]:
import hashlib
hashlib.sha256(b'111').digest()

b'\xf6\xe0\xa1\xe2\xacA\x94Z\x9a\xa7\xff\x8a\x8a\xaa\x0c\xeb\xc1*;\xcc\x98\x1a\x92\x9a\xd5\xcf\x81\n\t\x0e\x11\xae'

In [15]:
sha_256(b'111').hexdigest()

'f6e0a1e2ac41945a9aa7ff8a8aaa0cebc12a3bcc981a929ad5cf810a090e11ae'

In [16]:
hashlib.sha256(b'111').hexdigest()

'f6e0a1e2ac41945a9aa7ff8a8aaa0cebc12a3bcc981a929ad5cf810a090e11ae'